In [2]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.Collecting imutils
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25853 sha256=86ea500064d824e24a838dbe7fb6b6bdcc7aa3154cbd418dd851e38354752147
  Stored in directory: c:\users\nalin\appdata\local\pip\cache\wheels\5b\76\96\ad0c321506837bef578cf3008df3916c23018435a355d9f6b1
Successfully built imutils




[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install dlib


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
     --------------------------------------- 0.0/11.8 MB 487.6 kB/s eta 0:00:25
      --------------------------------------- 0.2/11.8 MB 1.4 MB/s eta 0:00:09
     - -------------------------------------- 0.3/11.8 MB 1.8 MB/s eta 0:00:07
     - -------------------------------------- 0.5/11.8 MB 2.2 MB/s eta 0:00:06
     -- ------------------------------------- 0.6/11.8 MB 2.5 MB/s eta 0:00:05
     -- ------------------------------------- 0.8/11.8 MB 2.6 MB/s eta 0:00:05
     --- ------------------------------------ 0.9/11.8 MB 2.7 MB/s eta 0:00:04
     --- ------------------------------------ 1.1/11.8 MB 2.8 MB/s eta 0:00:04
     ---- ----------------------------------- 1.3/11.8 MB 2.9 MB/s eta 0:00:04
     ---- ----------------------------------- 1.4/11.8 MB 2.8 MB/s eta 0:00:04
     ----- ---------------------------------- 1.6/11.8 MB 3.0 MB

In [5]:
pip install pyttsx3

  Obtaining dependency information for pyttsx3 from https://files.pythonhosted.org/packages/33/9a/de4781245f5ad966646fd276259ef7cfd400ba3cf7d5db7c0d5aab310c20/pyttsx3-2.90-py3-none-any.whl.metadata
  Obtaining dependency information for comtypes from https://files.pythonhosted.org/packages/2b/82/b897453a633dfd25bc09492e91b27f2f2fed5c9ef8d6d5fd7ecdb5148786/comtypes-1.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for pypiwin32 from https://files.pythonhosted.org/packages/d0/1b/2f292bbd742e369a100c91faa0483172cd91a1a422a6692055ac920946c5/pypiwin32-223-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/197.8 kB ? eta -:--:--
   ------------------ --------------------- 92.2/197.8 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 197.8/197.8 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# python drowniness_yawn.py --webcam webcam_index
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import imutils
import time
import dlib
import cv2
import pyttsx3

v = pyttsx3.init()
voices = v.getProperty('voices')
v.setProperty("rate", 178)
v.setProperty('voice', voices[1].id)


def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])

    ear = (A + B) / (2.0 * C)

    return ear


def final_ear(shape):
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]

    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)

    ear = (leftEAR + rightEAR) / 2.0
    return (ear, leftEye, rightEye)


def lip_distance(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))

    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))

    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)

    distance = abs(top_mean[1] - low_mean[1])
    return distance


EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 30
YAWN_THRESH = 14
alarm_status = False
alarm_status2 = False
saying = False
COUNTER = 0
YAWN_COUNTER = 0


detector = cv2.CascadeClassifier(r"E:\haarcascade_frontalface_default.xml")
predictor = dlib.shape_predictor(r"E:\shape_predictor_68_face_landmarks.dat")


vs = VideoStream(0).start()
# vs= VideoStream(usePiCamera=True).start()
time.sleep(1.0)

while True:

    frame = vs.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # rects = detector(gray, 0)
    rects = detector.detectMultiScale(gray, scaleFactor=1.1,
                                      minNeighbors=5, minSize=(30, 30),
                                      flags=cv2.CASCADE_SCALE_IMAGE)

    # for rect in rects:
    for (x, y, w, h) in rects:
        rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))

        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        eye = final_ear(shape)

        ear = eye[0]
        leftEye = eye[1]
        rightEye = eye[2]

        distance = lip_distance(shape)

        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        lip = shape[48:60]
        cv2.drawContours(frame, [lip], -1, (0, 255, 0), 1)

        if ear < EYE_AR_THRESH:
            COUNTER += 1

            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                v.say("Alert!!!! WAKE UP DUDE")
                v.runAndWait()
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

                

        else:
            COUNTER = 0
            

        if (distance > YAWN_THRESH):
            YAWN_COUNTER+=1
            if YAWN_COUNTER>=2:
                v.say("Stop the car")
                cv2.putText(frame, "Yawn Alert", (10, 130),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                YAWN_COUNTER = 0
                

        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "YAWN: {:.2f}".format(distance), (300, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "LEFTEYE: {:.2f}".format(eye_aspect_ratio(leftEye)), (0, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "RIGHTEYE: {:.2f}".format(eye_aspect_ratio(rightEye)), (0, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

cv2.destroyAllWindows()
vs.stop()

KeyboardInterrupt: 